<a href="https://colab.research.google.com/github/Shruteecodes04/celebal-assignments/blob/main/assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
full = pd.concat([train.drop('SalePrice', axis=1), test], axis=0)


In [2]:
missing = full.isnull().sum()
missing[missing > 0].sort_values(ascending=False)

,0
PoolQC,2909
MiscFeature,2814
Alley,2721
Fence,2348
MasVnrType,1766
FireplaceQu,1420
LotFrontage,486
GarageQual,159
GarageYrBlt,159
GarageCond,159


In [3]:
categorical_fill_none = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                         'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                         'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                         'MasVnrType']

numerical_fill_0 = ['GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2',
                    'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea']

for col in categorical_fill_none:
    full[col] = full[col].fillna("None")

for col in numerical_fill_0:
    full[col] = full[col].fillna(0)

# Fill remaining
full['Electrical'] = full['Electrical'].fillna(full['Electrical'].mode()[0])
full['LotFrontage'] = full.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median()))


In [4]:
full['TotalSF'] = full['TotalBsmtSF'] + full['1stFlrSF'] + full['2ndFlrSF']
full['TotalBath'] = full['FullBath'] + 0.5 * full['HalfBath'] + \
                    full['BsmtFullBath'] + 0.5 * full['BsmtHalfBath']
full['TotalPorchSF'] = full['OpenPorchSF'] + full['EnclosedPorch'] + \
                       full['3SsnPorch'] + full['ScreenPorch']


In [5]:
full['Age'] = full['YrSold'] - full['YearBuilt']
full['RemodAge'] = full['YrSold'] - full['YearRemodAdd']
full['GarageAge'] = full['YrSold'] - full['GarageYrBlt']



In [6]:
from sklearn.preprocessing import LabelEncoder

ordinal_features = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                    'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual',
                    'GarageCond', 'PoolQC']

for feature in ordinal_features:
    lbl = LabelEncoder()
    full[feature] = lbl.fit_transform(full[feature])


In [7]:
full = pd.get_dummies(full)

In [8]:

X_train = full[:train.shape[0]]
X_test = full[train.shape[0]:]
y_train = train['SalePrice']


In [9]:
import numpy as np
y_train = np.log1p(y_train)


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)